In [2]:
pip install keras

In [4]:
pip install tensorflow

In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding,LSTM

from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
from aranorm import normalize_arabic_text
from nltk.corpus import stopwords
from nltk import word_tokenize

import string

In [2]:
data = pd.read_excel('ALL_data.xlsx')
data = data.dropna()

In [3]:
data.columns = ['comment', 'Label']

In [4]:
data.head()

,comment,Label
0,نفسي يوم تكتبو السعر بدون مانسال,سلبي
1,طيب ما تشرحو طريقه الاشتراك في الباقه دي,محايد
2,لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...,سلبي
3,رمز الاشتراك شنو,محايد
4,واو,ايجابي


In [5]:
data.Label.unique()

array(['سلبي', 'محايد', 'ايجابي'], dtype=object)

In [6]:
data.shape

(3073, 2)

In [7]:
pos = []
neg = []
net = []
for l in data.Label:
    if l == 'سلبي':
        pos.append(0)
        neg.append(1)
        net.append(0)
    elif l == 'ايجابي':
        pos.append(1)
        neg.append(0)
        net.append(0)
    elif l == 'محايد':
        pos.append(0)
        neg.append(0)
        net.append(1)
data['Pos']= pos
data['Neg']= neg
data['Net']= net
data.head()

,comment,Label,Pos,Neg,Net
0,نفسي يوم تكتبو السعر بدون مانسال,سلبي,0,1,0
1,طيب ما تشرحو طريقه الاشتراك في الباقه دي,محايد,0,0,1
2,لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...,سلبي,0,1,0
3,رمز الاشتراك شنو,محايد,0,0,1
4,واو,ايجابي,1,0,0


In [8]:
data['clean'] = data['comment']

In [9]:
import stanza


nlp = stanza.Pipeline(lang='ar', processors='tokenize,lemma')


2023-01-06 08:48:58 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-06 08:48:59 WARNING: Language ar package default expects mwt, which has been added
2023-01-06 08:48:59 INFO: Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| lemma     | padt    |

2023-01-06 08:48:59 INFO: Use device: cpu
2023-01-06 08:48:59 INFO: Loading: tokenize
2023-01-06 08:48:59 INFO: Loading: mwt
2023-01-06 08:48:59 INFO: Loading: lemma
2023-01-06 08:48:59 INFO: Done loading processors!


In [10]:
arabicStopWords= stopwords.words("arabic")

for i in range(0,len(data)):
    tokenizedRow = word_tokenize(data['clean'][i])
    commentWithNoStopWords= ' '.join([i for i in tokenizedRow if i not in arabicStopWords])
                

    data['clean'][i]=commentWithNoStopWords
    

for i in range(0,len(data)):

    data['clean'][i]=normalize_arabic_text(data['clean'][i])

for i in range(0,len(data)):
    doc = nlp(data['clean'][i])
    lema=''.join(word.lemma+' ' for sent in doc.sentences for word in sent.words)

    data['clean'][i]=lema

for i in range(0,len(data)):  
    data['clean'][i]=normalize_arabic_text(data['clean'][i])

C:\Users\rabab\AppData\Local\Temp\ipykernel_5976\115673917.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean'][i]=commentWithNoStopWords
C:\Users\rabab\AppData\Local\Temp\ipykernel_5976\115673917.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean'][i]=normalize_arabic_text(data['clean'][i])
C:\Users\rabab\AppData\Local\Temp\ipykernel_5976\115673917.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['c

In [11]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.clean] 

In [12]:
result = [' '.join(sen) for sen in tokens] 

In [13]:
data['Text_Final'] = result
data['tokens'] = tokens
data = data[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg','Net']]

In [14]:
data[:4]

,Text_Final,tokens,Label,Pos,Neg,Net
0,نفسي يوم تكتب سعر بدون مانسال,"[نفسي, يوم, تكتب, سعر, بدون, مانسال]",سلبي,0,1,0
1,طيب تشرح طريق هو اشتراك باقه دي,"[طيب, تشرح, طريق, هو, اشتراك, باقه, دي]",محايد,0,0,1
2,قلل رساءل دي وادي لي هو ميقات كان اجمل,"[قلل, رساءل, دي, وادي, لي, هو, ميقات, كان, اجمل]",سلبي,0,1,0
3,رميز اشتراك شنا,"[رميز, اشتراك, شنا]",محايد,0,0,1


In [15]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

21186 words total, with a vocabulary size of 4197
Max sentence length is 79


In [16]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

2413 words total, with a vocabulary size of 957
Max sentence length is 64


In [19]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [21]:
# WORD2VEC

In [23]:
import tensorflow as tf


from numpy import array
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec

In [30]:
# prepare tokenizer

t = Tokenizer()
t.fit_on_texts(data['Text_Final'])
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(data['Text_Final'])
print('encoded_docs:\n',encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, 
padding='post')
print('padded_docs:\n',padded_docs)

encoded_docs:
 [[455, 15, 546, 9, 203, 1879], [134, 1880, 129, 1, 135, 174, 7], [1184, 547, 7, 1185, 26, 1, 1881, 38, 395], [859, 135, 24], [], [135], [7, 136, 4, 350, 267, 36, 1882, 204], [220, 92, 860, 8, 25, 7, 221, 66, 161, 1, 350, 205, 292, 1, 25], [75, 1883, 1884, 1885, 75, 3, 456, 247], [9], [154], [91, 673, 142, 2, 183, 861, 2, 174, 674, 1886, 1887, 1888, 26, 1, 674, 174, 268], [36, 184, 1186], [3, 72], [9, 61, 12, 1, 136], [25, 1, 62, 1, 1889, 1, 23, 862, 1, 51, 675, 1, 1890, 318, 863, 1, 676, 677], [396, 548, 546, 9, 122, 351, 206], [6, 162, 1, 52, 220, 207, 208, 36, 319, 2, 135, 1, 352, 14, 549, 550, 864, 26, 1, 397, 1, 865, 1891, 678, 1892, 865, 1893, 1, 1894, 1187], [23, 118, 94, 61, 679, 398, 2, 1895, 1, 2, 1896, 26, 61, 40, 21], [551, 457, 195, 1188, 2, 1189, 2, 1, 9, 209, 4], [4, 1897, 91, 1, 61, 183, 861, 1898, 1, 210, 123], [1190, 860, 353, 196, 64, 174, 36, 84], [6, 105, 41, 1, 54, 248, 866, 13, 269, 10], [211, 267, 25, 1, 552, 19, 267, 4, 458, 1, 53, 1, 2, 249, 84, 

In [32]:
# load the whole embedding into memory
w2v_embeddings_index={}
TOTAL_EMBEDDING_DIM=300
embeddings_file='…./full_grams_sg_300_twitter/full_grams_sg_300_twitter.mdl'
w2v_model =KeyedVectors.load(embeddings_file)
for word in w2v_model.wv.vocab:
    w2v_embeddings_index[word] = w2v_model[word]
print('Loaded %s word vectors.'% len(w2v_embeddings_index))

FileNotFoundError: [Errno 2] No such file or directory: '…./full_grams_sg_300_twitter/full_grams_sg_300_twitter.mdl'

In [33]:
!wget "https://bakrianoo.sfo2.digitaloceanspaces.com/aravec/full_grams_cbow_100_twitter.zip"
!unzip "full_grams_cbow_100_twitter.zip"

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
